# 1. 코드 기능 하나씩 구현

In [21]:
from product_name_preprocessing import *
product_name='[11월 올영픽/토너 250ml증정]바이오더마 하이드라비오 토너 500ml 기획(+토너 250ml 증정)'
text=preprocessing_product_name(product_name)
print(text)

바이오더마 하이드라비오 토너


In [26]:
from bs4 import BeautifulSoup
import requests


url=f'https://www.hwahae.co.kr/search?q={text}'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

a_tag=soup.select('#__next > main > section > section > div.px-20 > ul > li:nth-child(1) > div > a')
href=a_tag[0]['href']
print(href)



products/4900


In [5]:
https://www.hwahae.co.kr/products/4900                https://www.hwahae.co.kr/products/4900
https://www.hwahae.co.kr/goods/61961?goods_tab=review_ingredients          https://www.hwahae.co.kr/goods/61961?goods_tab=review_ingredients

SyntaxError: invalid syntax (629481214.py, line 1)

In [43]:
h={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
   'Cookie': '_ga_3V19ZGJ84W=GS1.1.1731286911.1.0.1731286911.0.0.0; _ga=GA1.1.1456301248.1731286912; _tt_enable_cookie=1; _ttp=J_Y3wZSgjwteQLHGjuCc8fENzKT; _fbp=fb.2.1731286926465.175496475186567195; aws-waf-token=5808b6ac-e973-45a5-b954-aa7fb0ec561e:AQoAYO8NypIXAAAA:kHHU5Y7cnBVv81SIvkgE4K61MLtfg/3BBBpU3kPu4aa+iB3RvGWaVRaFybyIjoqKS/OD9iiUUZY6uDLd5zwKs6WJhl+xsNqQORcTBdSvIBvFZB/OeHYp0YmsJURW/zJRuxbTcMve9Pw945FZ4crpCFLv6Gxk+9lFUhYGd3+jSHv/xHadA6USlvIp6eusn+K0RMeN; _ga_36NCBJ5CBH=GS1.1.1731289454.2.1.1731291782.60.0.1705907096'}


- 기능/효과 -> 화해 -> 수분, 진정, 미백, 탄력 -> 4개
- 제형 -> 화해 -> 가벼운, 무거운 -> 2개

In [ ]:
url2=f"https://www.hwahae.co.kr/{href}"
response2=requests.get(url2,headers=h)
soup2=BeautifulSoup(response2.text, 'html.parser')

good_review_tag=soup2.select_one('div.grow')
feature_tags = good_review_tag.select('ul li')

features = []
for tag in feature_tags:
    text_only = re.sub(r'\d+', '', tag.text)
    features.append(text_only.strip())

[<li class="flex justify-between gap-x-8"><span class="hds-text-caption-large text-gray-primary line-clamp-1">수분있는</span><span class="hds-text-body-medium text-gray-tertiary">762</span></li>, <li class="flex justify-between gap-x-8"><span class="hds-text-caption-large text-gray-primary line-clamp-1">보습잘되는</span><span class="hds-text-body-medium text-gray-tertiary">445</span></li>, <li class="flex justify-between gap-x-8"><span class="hds-text-caption-large text-gray-primary line-clamp-1">자극없는</span><span class="hds-text-body-medium text-gray-tertiary">310</span></li>, <li class="flex justify-between gap-x-8"><span class="hds-text-caption-large text-gray-primary line-clamp-1">흡수잘되는</span><span class="hds-text-body-medium text-gray-tertiary">192</span></li>, <li class="flex justify-between gap-x-8"><span class="hds-text-caption-large text-gray-primary line-clamp-1">속건조에효과있는</span><span class="hds-text-body-medium text-gray-tertiary">182</span></li>, <li class="flex justify-between gap-x-

In [58]:
features

['수분있는', '보습잘되는', '자극없는', '흡수잘되는', '속건조에효과있는', '향에만족한', '가벼운']

In [ ]:
# # 원본 리스트
# features = [
#     '수분있는', '보습잘되는', '자극없는', '흡수잘되는', '속건조에효과있는', '향에만족한',
#     '가벼운', '알러지반응오는', '흘러내리는', '인공향나는', '가려운', '냄새가독한',
#     '미끌거리는', '잘굳는'
# ]

# 카테고리 기준
categories = {
    '기능/효과': {
        '수분': ['흡수잘되는', '끈적하지않은', '속건조에효과있는', '보습잘되는', '수분있는'],
        '진정': ['트러블안생기는', '진정되는', '트러블없어지는', '자극없는', '가렵지않은', '뒤집어지지않는', '쿨링되는'],
        '미백': ['피부톤이개선되는', '각질제거잘되는'],
        '탄력': ['피부결좋아지는', '모공관리되는', '피부강화되는']
    },
    '제형': {
        '가벼운 제형': ['가벼운', '산뜻한', '끈적하지않은', '흡수잘되는'],
        '무거운 제형': ['쫀득한', '잘발리는', '리치한']
    }
}

# 분류 결과 저장용 사전, 기타 카테고리 포함
categorized_features = {
    '기능/효과': {
        '수분': [],
        '진정': [],
        '미백': [],
        '탄력': []
    },
    '제형': {
        '가벼운 제형': [],
        '무거운 제형': []
    },
    '기타': []  # 기타 카테고리 추가
}


# 분류 코드
for feature in features:
    categorized = False  # 해당 feature가 분류되었는지 여부 체크
    
    # 기능 카테고리 분류
    for subcategory, keywords in categories['기능/효과'].items():
        if feature in keywords:
            categorized_features['기능/효과'][subcategory].append(feature)
            categorized = True
            break

    # 제형 카테고리 분류
    if not categorized:  # 기능으로 분류되지 않은 경우에만 제형에서 검사
        for subcategory, keywords in categories['제형'].items():
            if feature in keywords:
                categorized_features['제형'][subcategory].append(feature)
                categorized = True
                break
    
    # '기타' 카테고리에 추가
    if not categorized:
        categorized_features['기타'].append(feature)

# 결과 출력
print(categorized_features)

{'기능/효과': {'수분': ['수분있는', '보습잘되는', '흡수잘되는', '속건조에효과있는'], '진정': ['자극없는'], '미백': [], '탄력': []}, '제형': {'가벼운 제형': ['가벼운'], '무거운 제형': []}, '기타': ['향에만족한']}


In [119]:
# df 형태로 받아오기

import pandas as pd

# 원본 리스트
# features = [
#     '수분있는', '보습잘되는', '자극없는', '흡수잘되는', '속건조에효과있는', '향에만족한',
#     '가벼운', '알러지반응오는', '흘러내리는', '인공향나는', '가려운', '냄새가독한',
#     '미끌거리는', '잘굳는'
# ]

# 카테고리 기준
categories = {
    '기능/효과': {
        '수분': ['흡수잘되는', '끈적하지않은', '속건조에효과있는', '보습잘되는', '수분있는'],
        '진정': ['트러블안생기는', '진정되는', '트러블없어지는', '자극없는', '가렵지않은', '뒤집어지지않는', '쿨링되는'],
        '미백': ['피부톤이개선되는', '각질제거잘되는'],
        '탄력': ['피부결좋아지는', '모공관리되는', '피부강화되는']
    },
    '제형': {
        '가벼운 제형': ['가벼운', '산뜻한', '끈적하지않은', '흡수잘되는'],
        '무거운 제형': ['쫀득한', '잘발리는', '리치한']
    }
}

# 분류 결과 저장용 사전
categorized_features = {
    '기능/효과': {
        '수분': [],
        '진정': [],
        '미백': [],
        '탄력': []
    },
    '제형': {
        '가벼운 제형': [],
        '무거운 제형': []
    }
}

# 분류 코드
for feature in features:
    categorized = False  # 해당 feature가 분류되었는지 여부 체크
    
    # 기능 카테고리 분류
    for subcategory, keywords in categories['기능/효과'].items():
        if feature in keywords:
            categorized_features['기능/효과'][subcategory].append(feature)
            categorized = True
            break

    # 제형 카테고리 분류
    if not categorized:  # 기능으로 분류되지 않은 경우에만 제형에서 검사
        for subcategory, keywords in categories['제형'].items():
            if feature in keywords:
                categorized_features['제형'][subcategory].append(feature)
                categorized = True
                break

# 기능/효과 및 제형 리스트 생성
function_list = [cat for cat, items in categorized_features['기능/효과'].items() if items]
formulation_list = [cat for cat, items in categorized_features['제형'].items() if items]

# DataFrame 생성
data = {
    'product_name': ['바이오더마'],
    'function': [function_list],
    'formulation': [formulation_list]
}
df = pd.DataFrame(data)

# 결과 출력
df

,product_name,function,formulation
0,바이오더마,[진정],[가벼운 제형]


# 2. py 파일로 구현

In [108]:
from bs4 import BeautifulSoup
import requests
import re

# 1. DB에서 제품명 가져오기
def preprocessing_product_name(text):
    patterns = [
        r'\[.*?\]|\(.*?\)|\d+ml.*'

    ]

    for pattern in patterns:
        text = re.sub(pattern, "", text)

    processed_text = re.sub(r'\s+', ' ', text).strip()
    return processed_text

# 2. 화해에서 제품명 검색하기
def get_product_href_tag(processed_text):
   source_url=f'https://www.hwahae.co.kr/search?q={processed_text}'
   response = requests.get(source_url)
   soup = BeautifulSoup(response.text, 'html.parser')

   a_tag=soup.select('#__next > main > section > section > div.px-20 > ul > li:nth-child(1) > div > a')
   href=a_tag[0]['href']
   # print(href) -> ex.) products/4900
   return href

# 3. 해당 제품의 AI 리뷰 데이터 추출하기
def get_product_ai_review(href):
   s = requests.Session()
   s.headers.update(
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
            "Cookie": "_ga_3V19ZGJ84W=GS1.1.1731286911.1.0.1731286911.0.0.0; _ga=GA1.1.1456301248.1731286912; _tt_enable_cookie=1; _ttp=J_Y3wZSgjwteQLHGjuCc8fENzKT; _fbp=fb.2.1731286926465.175496475186567195; aws-waf-token=5808b6ac-e973-45a5-b954-aa7fb0ec561e:AQoAhXQjLjqzAAAA:YV2/E4qkeGZhD2EZQlq5fsBhU3ZNyWVYfHw6tyh2h6DB3SAuTom2b/E4Re9zRYL7wPd9El9bg/pF+p9bDgp8HbPdKVaLBo4qBecA/eNYiQIOXkkxuNjKv4A1d4lJfLB6MtARVfnVuqTv3i7rZca8/Xyz18oETRzJiqTvTE8PuIYhZAlSu0DqeR/9kBsoGlc2Fv1g; _ga_36NCBJ5CBH=GS1.1.1731301644.4.1.1731302309.59.0.1961747604; CHECK_IS_REPRESENT_GOODS_42788=true",
        }
    )

   product_url=f"https://www.hwahae.co.kr/{href}"
   response2=s.get(product_url)
   soup2=BeautifulSoup(response2.text, 'html.parser')

   tags=soup2.select('div.grow ul li')

   features = []
   for tag in tags:
        text_only = re.sub(r'\d+', '', tag.text)
        features.append(text_only.strip())

   return features


In [109]:
product_name = '[11월 올영픽/토너 250ml증정]바이오더마 하이드라비오 토너 500ml 기획(+토너 250ml 증정)'
text = preprocessing_product_name(product_name)
href = get_product_href_tag(text)
features_test = get_product_ai_review(href)
print(features_test)

[]


In [136]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# 1. DB에서 제품명 가져오기
def preprocessing_product_name(text):
    patterns = [
        r'\[.*?\]|\(.*?\)|\d+ml.*'

    ]

    for pattern in patterns:
        text = re.sub(pattern, "", text)

    processed_text = re.sub(r'\s+', ' ', text).strip()
    return processed_text

# 2. 화해에서 제품명 검색하기
def get_product_href_tag(processed_text):
   source_url=f'https://www.hwahae.co.kr/search?q={processed_text}'
   response = requests.get(source_url)
   soup = BeautifulSoup(response.text, 'html.parser')

   a_tag=soup.select('#__next > main > section > section > div.px-20 > ul > li:nth-child(1) > div > a')
   href=a_tag[0]['href']
   return href

# 3. 해당 제품의 AI 리뷰 데이터 추출하기
def get_product_ai_review(href):
   s = requests.Session()
   s.headers.update(
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
            "Cookie": "_ga_3V19ZGJ84W=GS1.1.1731286911.1.0.1731286911.0.0.0; _ga=GA1.1.1456301248.1731286912; _tt_enable_cookie=1; _ttp=J_Y3wZSgjwteQLHGjuCc8fENzKT; _fbp=fb.2.1731286926465.175496475186567195; COOKIE_HIDE_GO_TO_APP_BOTTOM_SHEET=true; aws-waf-token=5808b6ac-e973-45a5-b954-aa7fb0ec561e:AQoAnC8zOB89AQAA:PAThP7dXUAkqH3D+BccMc4yEIuMx9jTxunsZWshwdUiJgOstFzXpPjEipIyLpRSh6tmXuC4Ily9YjjwxikgYLxk/L/mgGutxUQO6Qmrci/+sBQO2GBLC2q6e7XCW37TyyWLdFtJd667GhwcQu7cvK41YF/rJXWuNNMD3M6P2Vrl0b8im0GoshJ2UXfUD6nXhLJfU; _ga_36NCBJ5CBH=GS1.1.1731306072.5.1.1731310770.60.0.1372893092",
        }
    )

   product_url=f"https://www.hwahae.co.kr/{href}"
   print(product_url)
   response2=s.get(product_url)
   print(response2)
   soup2=BeautifulSoup(response2.text, 'html.parser')

   good_review_tag=soup2.select_one('div.grow')
   feature_tags = good_review_tag.select('ul li')

   features = []
   for tag in feature_tags:
        text_only = re.sub(r'\d+', '', tag.text)
        features.append(text_only.strip())

   return features

# 4. 제품에 대한 features를 카테고리화
def categorize_features(features):
    # 카테고리 기준
    categories = {
        '기능/효과': {
            '수분': ['흡수잘되는', '끈적하지않은', '속건조에효과있는', '보습잘되는', '수분있는'],
            '진정': ['트러블안생기는', '진정되는', '트러블없어지는', '자극없는', '가렵지않은', '뒤집어지지않는', '쿨링되는'],
            '미백': ['피부톤이개선되는', '각질제거잘되는'],
            '탄력': ['피부결좋아지는', '모공관리되는', '피부강화되는']
        },
        '제형': {
            '가벼운 제형': ['가벼운', '산뜻한', '끈적하지않은', '흡수잘되는'],
            '무거운 제형': ['쫀득한', '잘발리는', '리치한']
        }
    }

    # 분류 결과 저장용 사전
    categorized_features = {
        '기능/효과': {subcategory: [] for subcategory in categories['기능/효과']},
        '제형': {subcategory: [] for subcategory in categories['제형']}
    }

    # 분류 코드
    for feature in features:
        categorized = False  # 해당 feature가 분류되었는지 여부 체크
        
        # 기능 카테고리 분류
        for subcategory, keywords in categories['기능/효과'].items():
            if feature in keywords:
                categorized_features['기능/효과'][subcategory].append(feature)
                categorized = True
                break

        # 제형 카테고리 분류
        if not categorized:  # 기능으로 분류되지 않은 경우에만 제형에서 검사
            for subcategory, keywords in categories['제형'].items():
                if feature in keywords:
                    categorized_features['제형'][subcategory].append(feature)
                    categorized = True
                    break

    # 기능/효과 및 제형 리스트 생성
    function_list = [cat for cat, items in categorized_features['기능/효과'].items() if items]
    formulation_list = [cat for cat, items in categorized_features['제형'].items() if items]

    return function_list, formulation_list

# 5. Processing the CSV file in chunks and saving results
def process_products_from_csv(input_csv, output_prefix, chunk_size=20):
    products_df = pd.read_csv(input_csv)
    num_chunks = (len(products_df) // chunk_size) + (1 if len(products_df) % chunk_size else 0)

    for i in range(num_chunks):
        chunk_df = products_df.iloc[i * chunk_size: (i + 1) * chunk_size]
        result_data = []

        for _, row in chunk_df.iterrows():
            product_name = row['product_name']
            processed_name = preprocessing_product_name(product_name)

            href = get_product_href_tag(processed_name)
            if href:
                features = get_product_ai_review(href)
                function_list, formulation_list = categorize_features(features)
                result_data.append({
                    'product_name': processed_name,
                    'function': function_list,
                    'formulation': formulation_list
                })
            else:
                print(f"Product {processed_name} not found on 화해.")

        # Save each chunk result to a separate CSV file
        output_csv = f"{output_prefix}_{i + 1}.csv"
        result_df = pd.DataFrame(result_data)
        result_df.to_csv(output_csv, index=False)
        print(f"Results saved to {output_csv}")

# Execute the process
if __name__ == "__main__":
    input_csv = 'unique_products.csv'
    output_prefix = '스킨토너_결과'
    process_products_from_csv(input_csv, output_prefix)


https://www.hwahae.co.kr/products/4900
<Response [200]>
https://www.hwahae.co.kr/goods/61961?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/goods/48283?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/products/2115786
<Response [200]>
https://www.hwahae.co.kr/goods/48436?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/goods/61961?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/goods/56188?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/goods/57154?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/goods/26143?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/goods/12318?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/goods/47881?goods_tab=review_ingredients
<Response [200]>
https://www.hwahae.co.kr/products/2044266
<Response [200]>
https://www.hwahae.co.kr/goods/35549?goods_tab=review_ingredients
<Response [20

AttributeError: 'NoneType' object has no attribute 'select'

In [126]:
df

,product_name,function,formulation
0,피지오겔 DMT 페이셜 로션,[진정],[가벼운 제형]


In [ ]:


# 사용법 예시
# python hwahwe_crawl.py [변우석pick] 피지오겔 DMT 페이셜 로션 200ml+50ml+포토스티커 한정기획

In [130]:
import pandas as pd

# CSV 파일을 DataFrame으로 불러오기
df = pd.read_csv('스킨토너.csv')

# product_name 값이 같은 경우 첫 번째 행만 남기고 중복 제거
df_unique = df.drop_duplicates(subset='product_name', keep='first')

# 결과를 새로운 CSV 파일로 저장하기 (필요에 따라 경로와 파일 이름 변경)
df_unique.to_csv('unique_products.csv', index=False)